In [6]:
import sys
sys.path.append('..')

from utils.misc import *
from utils.classifier import *
from utils.visualiser import *

## Loading data

In [9]:
# train
df_train = pd.read_csv("../data/train/df_train.csv")

df_train_b0 = pd.read_csv("../data/train/df_train_b0.csv")
df_train_b1 = pd.read_csv("../data/train/df_train_b1.csv")
df_train_b2 = pd.read_csv("../data/train/df_train_b2.csv")

df_train_b0_f0 = pd.read_csv("../data/train/df_train_b0_f0.csv")
df_train_b0_f1 = pd.read_csv("../data/train/df_train_b0_f1.csv")
df_train_b0_f2 = pd.read_csv("../data/train/df_train_b0_f2.csv")
df_train_b0_f3 = pd.read_csv("../data/train/df_train_b0_f3.csv")

df_train_b1_f0 = pd.read_csv("../data/train/df_train_b1_f0.csv")
df_train_b1_f1 = pd.read_csv("../data/train/df_train_b1_f1.csv")
df_train_b1_f2 = pd.read_csv("../data/train/df_train_b1_f2.csv")
df_train_b1_f3 = pd.read_csv("../data/train/df_train_b1_f3.csv")

df_train_b2_f0 = pd.read_csv("../data/train/df_train_b2_f0.csv")
df_train_b2_f1 = pd.read_csv("../data/train/df_train_b2_f1.csv")
df_train_b2_f2 = pd.read_csv("../data/train/df_train_b2_f2.csv")
df_train_b2_f3 = pd.read_csv("../data/train/df_train_b2_f3.csv")
df_train_b2_f4 = pd.read_csv("../data/train/df_train_b2_f4.csv")

df_train_wap = np.load("../data/train/df_train_wap.npy", allow_pickle=True)

df_train_b0_wap = np.load("../data/train/df_train_b0_wap.npy", allow_pickle=True)
df_train_b1_wap = np.load("../data/train/df_train_b1_wap.npy", allow_pickle=True)
df_train_b2_wap = np.load("../data/train/df_train_b2_wap.npy", allow_pickle=True)

df_train_b0_f0_wap = np.load("../data/train/df_train_b0_f0_wap.npy", allow_pickle=True)
df_train_b0_f1_wap = np.load("../data/train/df_train_b0_f1_wap.npy", allow_pickle=True)
df_train_b0_f2_wap = np.load("../data/train/df_train_b0_f2_wap.npy", allow_pickle=True)
df_train_b0_f3_wap = np.load("../data/train/df_train_b0_f3_wap.npy", allow_pickle=True)

df_train_b1_f0_wap = np.load("../data/train/df_train_b1_f0_wap.npy", allow_pickle=True)
df_train_b1_f1_wap = np.load("../data/train/df_train_b1_f1_wap.npy", allow_pickle=True)
df_train_b1_f2_wap = np.load("../data/train/df_train_b1_f2_wap.npy", allow_pickle=True)
df_train_b1_f3_wap = np.load("../data/train/df_train_b1_f3_wap.npy", allow_pickle=True)

df_train_b2_f0_wap = np.load("../data/train/df_train_b2_f0_wap.npy", allow_pickle=True)
df_train_b2_f1_wap = np.load("../data/train/df_train_b2_f1_wap.npy", allow_pickle=True)
df_train_b2_f2_wap = np.load("../data/train/df_train_b2_f2_wap.npy", allow_pickle=True)
df_train_b2_f3_wap = np.load("../data/train/df_train_b2_f3_wap.npy", allow_pickle=True)
df_train_b2_f4_wap = np.load("../data/train/df_train_b2_f4_wap.npy", allow_pickle=True)